In [1]:
import numpy as np
import pandas as pd
import shap
import sklearn
import time

In [ ]:
#CatBoost
from catboost import CatBoostClassifier
def catb(X, y, index):
    model = CatBoostClassifier(random_state=2025, verbose=False)
    model.fit(X, y)
    explainer = shap.TreeExplainer(model)
    X_background = X.iloc[index]
    shap_values = explainer(X_background)
    importance = shap_values.abs.mean(axis=0).values
    return importance

In [ ]:
#LightGBM
from lightgbm import LGBMClassifier
def ligb(X, y, index):
    model = LGBMClassifier(random_state=2025)
    model.fit(X, y)
    explainer = shap.TreeExplainer(model)
    X_background = X.iloc[index]
    shap_values = explainer(X_background)
    importance = shap_values.abs.mean(axis=0).values
    return importance

In [ ]:
#XGBoost
from xgboost import XGBClassifier
def xgb(X, y, index):
    model = XGBClassifier(random_state=2025, eval_metric='logloss')
    model.fit(X, y)
    explainer = shap.TreeExplainer(model)
    X_background = X.iloc[index]
    shap_values = explainer(X_background)
    importance = shap_values.abs.mean(axis=0).values
    return importance

In [ ]:
#GBDT
from sklearn.ensemble import GradientBoostingClassifier
def gbdt(X, y, index):
    model = GradientBoostingClassifier(random_state=2025)
    model.fit(X, y)
    explainer = shap.Explainer(model)
    X_background = X.iloc[index]
    shap_values = explainer(X_background)
    importance = shap_values.abs.mean(axis=0).values
    return importance

In [2]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
def rf(X, y, index):
    rmodelf = RandomForestClassifier(random_state=2025)
    model.fit(X, y)
    explainer = shap.TreeExplainer(model)
    X_background = X.iloc[index]
    shap_values = explainer(X_background)
    importance = shap_values.abs.mean(axis=0).values[:,0]
    return importance

In [ ]:
X = pd.read_csv("data matrix X")
y = pd.read_csv("outcome y")
index = pd.read_csv("index for subset to calculate shap values")
M=5 #number of knockoffs

start_time = time.time()

ncol = X.shape[1]
rf_q1 = rf(X,y,index)
rf_im = pd.DataFrame(np.zeros((M+1, ncol)))
rf_im.iloc[0] = rf_q1
    
catb_q1 = catb(X,y,index)
catb_im = pd.DataFrame(np.zeros((M+1, ncol)))
catb_im.iloc[0] = catb_q1

xgb_q1 = xgb(X,y,index)
xgb_im = pd.DataFrame(np.zeros((M+1, ncol)))
xgb_im.iloc[0] = xgb_q1

ligb_q1 = ligb(X,y,index)
ligb_im = pd.DataFrame(np.zeros((M+1, ncol)))
ligb_im.iloc[0] = ligb_q1

gbdt_q1 = gbdt(X,y,index)
gbdt_im = pd.DataFrame(np.zeros((M+1, ncol)))
gbdt_im.iloc[0] = gbdt_q1


    
print("Original, Done!")

for i in range(M):
    
    file = f'knockoff data matrix X_i'  
    
    data = pd.read_csv(file)
        
    rf_im.iloc[i+1] = rf(data,y,index)
    catb_im.iloc[i+1] = catb(data,y,index)
    xgb_im.iloc[i+1] = xgb(data,y,index)
    ligb_im.iloc[i+1] = ligb(data,y,index)
    gbdt_im.iloc[i+1] = gbdt(data,y,index)
    
    
    print(f"Index: {i + 1}, Done!")
        
    
basepath = ""    
file_rf = f"{basepath}/rf_fi.csv"
file_catb = f"{basepath}/catb_fi.csv"
file_xgb = f"{basepath}/xgb_fi.csv"
file_ligb = f"{basepath}/ligb_fi.csv"
file_gbdt = f"{basepath}/gbdt_fi.csv"
    
rf_im.to_csv(file_rf)
catb_im.to_csv(file_catb)
xgb_im.to_csv(file_xgb)
ligb_im.to_csv(file_ligb)
gbdt_im.to_csv(file_gbdt)

end_time = time.time()
elapsed_time = end_time - start_time

elapsed_minutes = elapsed_time / 60
print(f"Total running time: {elapsed_minutes:.2f} minutes")